<a href="https://colab.research.google.com/github/cuongtm2/machine-learning/blob/master/TF_training_Day_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import csv
import tensorflow as tf
import numpy as np
import urllib
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Build and train a neural network to predict sunspot activity using
# the Sunspots.csv dataset.
#
# Your neural network must have an MAE of 0.12 or less on the normalized dataset
# for top marks.
#
# Code for normalizing the data is provided and should not be changed.
#
# At the bottom of this file, we provide  some testing
# code in case you want to check your model.

# Note: Do not use lambda layers in your model, they are not supported
# on the grading infrastructure.

In [20]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/Sunspots.csv'
urllib.request.urlretrieve(url, 'sunspots.csv')

time_step = []
sunspots = []

with open('sunspots.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        sunspots.append(float(row[2]))
        time_step.append(int(row[0]))

series = np.array(sunspots)

In [3]:
# This is the normalization function
min = np.min(series)
max = np.max(series)
series -= min
series /= max
time = np.array(time_step)

# The data should be split into training and validation sets at time step 3000
# DO NOT CHANGE THIS CODE
split_time = 3000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

In [21]:
# mean = np.mean(series)
# std = np.std(series)
# series -= mean
# series /= std
# time = np.array(time_step)

# # The data should be split into training and validation sets at time step 3000
# # DO NOT CHANGE THIS CODE
# split_time = 3000
# time_train = time[:split_time]
# x_train = series[:split_time]
# time_valid = time[split_time:]
# x_valid = series[split_time:]

In [22]:
# DO NOT CHANGE THIS CODE
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

In [23]:
# Parameter 1
window_size = 30
batch_size = 32
shuffle_buffer_size = 1000
train_set = windowed_dataset(x_train, window_size=window_size, batch_size=batch_size,
                              shuffle_buffer=shuffle_buffer_size)
val_set = windowed_dataset(x_valid, window_size=window_size, batch_size=batch_size,
                              shuffle_buffer=shuffle_buffer_size)


# Parameter 2
# window_size = 100
# batch_size = 512
# shuffle_buffer_size = 1000
# train_set = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

In [24]:
tf.keras.backend.clear_session()

# Model 2
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(filters=64, kernel_size=5, strides=1, padding='causal', activation='relu', input_shape=[None, 1]),
  tf.keras.layers.Bidirectional(LSTM(64, return_sequences=True)),
  tf.keras.layers.Bidirectional(LSTM(64, return_sequences=True)),
  tf.keras.layers.Dense(64, activation="relu"),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(32, activation="relu"),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(16, activation="relu"),
  tf.keras.layers.Dense(1)
  # tf.keras.layers.Lambda(lambda x: x * 400)
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, None, 64)          384       
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        66048     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, None, 128)        98816     
 nal)                                                            
                                                                 
 dense (Dense)               (None, None, 64)          8256      
                                                                 
 dropout (Dropout)           (None, None, 64)          0         
                                                                 
 dense_1 (Dense)             (None, None, 32)          2

In [13]:
patient = 10
save_path = '/content/drive/MyDrive/tf_certification/model5.h5'
callbacks_list = [
    EarlyStopping(
        monitor='val_loss', 
        patience=patient, 
        mode='min', 
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor = 'val_loss', 
        factor = 0.5, 
        patience = patient / 2, 
        min_lr=1e-6,
        verbose=1,
        mode='min'
    ),
    ModelCheckpoint(save_path, 
                    monitor="val_mae",
                    mode="min",
                    verbose=1,
                    save_best_only=True)
    ]

In [15]:
# Training model 2 - parameter 1
model.compile(loss=tf.keras.losses.Huber(), optimizer='adam', metrics=['mae'])
history = model.fit(train_set, epochs=200, verbose=1,validation_data=val_set,
                    callbacks=callbacks_list)

Epoch 1/200
     93/Unknown - 16s 81ms/step - loss: 0.0058 - mae: 0.0752
Epoch 1: val_mae improved from inf to 0.05803, saving model to /content/drive/MyDrive/tf_certification/model5.h5
93/93 [==============================] - 18s 108ms/step - loss: 0.0058 - mae: 0.0752 - val_loss: 0.0032 - val_mae: 0.0580 - lr: 0.0010
Epoch 2/200
93/93 [==============================] - ETA: 0s - loss: 0.0028 - mae: 0.0536
Epoch 2: val_mae improved from 0.05803 to 0.05701, saving model to /content/drive/MyDrive/tf_certification/model5.h5
93/93 [==============================] - 8s 90ms/step - loss: 0.0028 - mae: 0.0536 - val_loss: 0.0033 - val_mae: 0.0570 - lr: 0.0010
Epoch 3/200
93/93 [==============================] - ETA: 0s - loss: 0.0022 - mae: 0.0474
Epoch 3: val_mae improved from 0.05701 to 0.05276, saving model to /content/drive/MyDrive/tf_certification/model5.h5
93/93 [==============================] - 8s 85ms/step - loss: 0.0022 - mae: 0.0474 - val_loss: 0.0031 - val_mae: 0.0528 - lr: 0.0010

In [18]:
# THIS CODE IS USED IN THE TESTER FOR FORECASTING. IF YOU WANT TO TEST YOUR MODEL
# BEFORE UPLOADING YOU CAN DO IT WITH THIS
model = tf.keras.models.load_model(save_path)
import math
def model_forecast(model, series, window_size):
   ds = tf.data.Dataset.from_tensor_slices(series)
   ds = ds.window(window_size, shift=1, drop_remainder=True)
   ds = ds.flat_map(lambda w: w.batch(window_size))
   ds = ds.batch(32).prefetch(1)
   forecast = model.predict(ds)
   return forecast

split_time = 3000
window_size = 30
# YOUR CODE HERE
rnn_forecast = model_forecast(model, series[..., np.newaxis], window_size)
rnn_forecast = rnn_forecast[split_time - window_size:-1, -1, 0]

result = tf.keras.metrics.mean_absolute_error(x_valid, rnn_forecast).numpy()

# To get the maximum score, your model must have an MAE OF .12 or less.
# When you Submit and Test your model, the grading infrastructure
# converts the MAE of your model to a score from 0 to 5 as follows:

test_val = 100 * result
score = math.ceil(17 - test_val)
if score > 5:
   score = 5

print(score)

5


In [25]:
patient = 10
save_path = '/content/drive/MyDrive/tf_certification/model5_1.h5'
callbacks_list = [
    EarlyStopping(
        monitor='val_loss', 
        patience=patient, 
        mode='min', 
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor = 'val_loss', 
        factor = 0.5, 
        patience = patient / 2, 
        min_lr=1e-6,
        verbose=1,
        mode='min'
    ),
    ModelCheckpoint(save_path, 
                    monitor="val_mae",
                    mode="min",
                    verbose=1,
                    save_best_only=True)
    ]

In [26]:
# Training model 2 - parameter 1
model.compile(loss=tf.keras.losses.Huber(), optimizer='adam', metrics=['mae'])
history = model.fit(train_set, epochs=200, verbose=1,validation_data=val_set,
                    callbacks=callbacks_list)

Epoch 1/200
     93/Unknown - 16s 80ms/step - loss: 0.1879 - mae: 0.4942
Epoch 1: val_mae improved from inf to 0.29265, saving model to /content/drive/MyDrive/tf_certification/model5_1.h5
93/93 [==============================] - 18s 106ms/step - loss: 0.1879 - mae: 0.4942 - val_loss: 0.0775 - val_mae: 0.2927 - lr: 0.0010
Epoch 2/200
93/93 [==============================] - ETA: 0s - loss: 0.0753 - mae: 0.2920
Epoch 2: val_mae improved from 0.29265 to 0.24257, saving model to /content/drive/MyDrive/tf_certification/model5_1.h5
93/93 [==============================] - 8s 87ms/step - loss: 0.0753 - mae: 0.2920 - val_loss: 0.0618 - val_mae: 0.2426 - lr: 0.0010
Epoch 3/200
93/93 [==============================] - ETA: 0s - loss: 0.0554 - mae: 0.2461
Epoch 3: val_mae did not improve from 0.24257
93/93 [==============================] - 8s 83ms/step - loss: 0.0554 - mae: 0.2461 - val_loss: 0.0809 - val_mae: 0.2612 - lr: 0.0010
Epoch 4/200
93/93 [==============================] - ETA: 0s - los

In [27]:
# THIS CODE IS USED IN THE TESTER FOR FORECASTING. IF YOU WANT TO TEST YOUR MODEL
# BEFORE UPLOADING YOU CAN DO IT WITH THIS
model = tf.keras.models.load_model(save_path)
import math
def model_forecast(model, series, window_size):
   ds = tf.data.Dataset.from_tensor_slices(series)
   ds = ds.window(window_size, shift=1, drop_remainder=True)
   ds = ds.flat_map(lambda w: w.batch(window_size))
   ds = ds.batch(32).prefetch(1)
   forecast = model.predict(ds)
   return forecast

split_time = 3000
window_size = 30
# YOUR CODE HERE
rnn_forecast = model_forecast(model, series[..., np.newaxis], window_size)
rnn_forecast = rnn_forecast[split_time - window_size:-1, -1, 0]

result = tf.keras.metrics.mean_absolute_error(x_valid, rnn_forecast).numpy()

# To get the maximum score, your model must have an MAE OF .12 or less.
# When you Submit and Test your model, the grading infrastructure
# converts the MAE of your model to a score from 0 to 5 as follows:

test_val = 100 * result
score = math.ceil(17 - test_val)
if score > 5:
   score = 5

print(score)

-15
